In [ ]:
import numpy as np
from numpy.linalg import inv
import time
import scipy as sc
import matplotlib.pyplot as plt
import cmath as cm
%matplotlib inline

ti=time.clock()

## Constants and parameters

In [ ]:
T = 0           #temperature
Γ = 100         #circulation scaled by 10**9

N=100

## Functions & Equations

In [ ]:
# INITIAL VORTEX LINE PARAMETRIZATION

#all distances in Angstroms

R = 3                #ring radius
X0 = Y0 = Z0 = 5     #ring center coordinates
r = np.array([X0,Y0,Z0,0,0])


#INITIAL POSTIONS

s0 = np.zeros([N,5])
ϕ = 2*np.pi/N        #infinitesimal angle
for i in range (1,N-1):
    s0[i,:] = r + [0,R*np.sin(i*ϕ),R*np.cos(i*ϕ), i-1, i+1]
s0[0,:] = r + [0,R*np.sin(0),R*np.cos(0), N-1, 1]
s0[N-1,:] = r + [0,R*np.sin((N-1)*ϕ),R*np.cos((N-1)*ϕ), N-2, 0]
# this will be much complicated due to the searching for neighbours = shortest path from i to j.

pos = s0[:,0:3]
next = s0[:,4]


# FINITE DIFFERENCE CALCULATOR

def der(s, atom, order, area):
    
    d = (area-1)/2 #points on one side from the center one
    
    M = np.zeros([area,area])
    koef = np.zeros([area])
    vector = np.zeros([area])
    
    for j in range (0,area):
        
        neigh = atom + (j-d)
        if neigh<0:
            neigh = N + neigh
        if neigh>(N-1):
            neigh = neigh - N
        
        h = np.linalg.norm(pos[atom] - pos[neigh])
        
        for i in range (0,area):
            M[i,j] = (-h)**i
    
    vector[order] = np.math.factorial(order)
    
    invM = np.linalg.solve(M, np.eye(area))
    print(M)
    print(np.linalg.det(M))
    print(invM)
    print(np.dot(M,invM))
    koef = np.dot(invM,vector)
    
    return koef



def s_der(s, atom, area):
    order = 1
    
    s_der = np.zeros([3])
    koef = der(s, atom, order, area)
    print(koef)
    d = (area-1)/2 #points on one side from the center one
    
    for i in range (0,area):
        neigh = atom+(i-d)
        
        if neigh<0:
            neigh = N + neigh
        if neigh>(N-1):
            neigh = neigh - N
            
        s_der += koef[i]*pos[neigh,0:3]
        
    return s_der
    
    

# LIA velocity in nm/s

def v_lia(s):
    
    v_lia = np.zeros((N,3))
    a = 1
    
    for i in range (0,N):
        r = 1/np.linalg.norm(s_der2(s)[i,:])
        β = (Γ*np.log(r/a))/(4*np.pi)
        v_lia[i,:] = β*np.cross(s_der(s)[i,:], s_der2(s)[i,:])
        
    return v_lia
    
    

# Biot-Savart for velocity field

def vel_i(r,s):
    vel_i = np.zeros(3)
    vel_i = Γ/(4*π) #*Biotsavart integral
    return vel_i


## Arrays

In [ ]:
der(s0, 1, 1, 3)

In [ ]:
np.linalg.norm(pos[2], pos[3])

## Playground

In [ ]:
np.eye(5)

## GARBAGE

In [ ]:
# DERIVATIVES

def s_der(s):
    s_der = np.zeros([N,3])
    
    s_der[0,:] = (pos[1,:] - pos[N-1,:])/(ϕ*R)
    
    for i in range (1,N-1):
        s_der[i,:] = (pos[i+1,:] - pos[i-1,:])/(ϕ*R)
    
    s_der[N-1,:] = (pos[0,:] - pos[N-2,:])/(ϕ*R)
    
    return s_der
    #this is only simple version for ring
    
# SECOND DERIVATIVES

def s_der2(s):
    s_der2 = np.zeros([N,3])
    
    s_der2[0,:] = (pos[2,:] - 2*pos[0,:] + pos[N-2,:])/(ϕ*R)**2
    s_der2[1,:] = (pos[3,:] - 2*pos[1,:] + pos[N-1,:])/(ϕ*R)**2
    
    for i in range (2,N-2):
        s_der2[i,:] = (pos[i+2,:] - 2*pos[i,:] + pos[i-2,:])/(ϕ*R)**2
    
    s_der2[N-1,:] = (pos[1,:] - 2*pos[N-1,:] + pos[N-3,:])/(ϕ*R)**2
    s_der2[N-2,:] = (pos[0,:] - 2*pos[N-2,:] + pos[N-4,:])/(ϕ*R)**2
    
    eps = 10e-5
    
    for i in range (0,N):
        for j in range (0,3):
            if np.absolute(s_der2[i,j]) < eps:
                s_der2[i,j] = 0
    
    return s_der2

#ARCLENGTH

def ξ(s):
    ξ = np.zeros([N])
    ξ[0] = 0
    for i in range(1,N):
        ξ[i] = ξ[i-1] + np.linalg.norm(next[np.where(next==i)] - next[np.where(next==(i-1))])
    
    return ξ
